<a href="https://colab.research.google.com/github/macroact-lab/robotics-book/blob/main/6_2_grid_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from matplotlib.colors import ListedColormap
import time

In [ ]:
class GridWorldEnv:
    def __init__(self, size=5):
        self.size = size
        self.actions = ["up", "right", "down", "left"]
        self.grid = np.zeros((size, size))

        # 장애물(1)과 목적지(2) 배치
        self.grid[1, 1] = 1
        self.grid[2, 1] = 1
        self.grid[1, 3] = 1
        self.grid[3, 3] = 1
        self.grid[size-1, size-1] = 2
        self.reset()

    def reset(self):
        """환경 초기화: 로봇을 시작점(0,0)으로 보낸다."""
        self.agent_pos = [0, 0]
        # (관측값, 추가정보)를 반환하는 표준 형식을 따른다.
        return tuple(self.agent_pos), {}

    def step(self, action_idx):
        """에이전트의 행동을 받아 상태를 변화시킨다."""
        action = self.actions[action_idx]
        old_pos = self.agent_pos.copy()

        # 1. 위치 이동 계산
        if action == "up" and self.agent_pos[0] > 0:
            self.agent_pos[0] -= 1
        elif action == "right" and self.agent_pos[1] < self.size - 1:
            self.agent_pos[1] += 1
        elif action == "down" and self.agent_pos[0] < self.size - 1:
            self.agent_pos[0] += 1
        elif action == "left" and self.agent_pos[1] > 0:
            self.agent_pos[1] -= 1

        # 2. 결과(보상/종료여부) 판정
        terminated = False
        if self.grid[self.agent_pos[0], self.agent_pos[1]] == 1: # 장애물 충돌
            self.agent_pos = old_pos # 뒤로가기
            reward = -1
        elif self.grid[self.agent_pos[0], self.agent_pos[1]] == 2: # 목적지
            reward = 10
            terminated = True
        else: # 일반 이동 (최단 경로 유도를 위해 작은 페널티)
            reward = -0.1

        # 표준 반환: 다음상태, 보상, 종료, 중단(여기선 사용X), 정보
        return tuple(self.agent_pos), reward, terminated, False, {}

    def render(self, q_table=None):
        """로봇의 위치와 지능(Q-table)을 화면에 그린다."""
        vis_grid = self.grid.copy()
        vis_grid[self.agent_pos[0], self.agent_pos[1]] = 3
        colors = ['white', 'black', 'green', 'red']
        cmap = ListedColormap(colors)
        plt.figure(figsize=(6, 5))
        sns.heatmap(vis_grid, cmap=cmap, vmin=0, vmax=3, cbar=False, linewidths=1, linecolor='black', square=True)
        plt.title("Grid World Navigation")
        plt.show(block=False)
        plt.pause(0.1)
        plt.close()

In [ ]:
class QLearningAgent:
    def __init__(self, state_size, action_size, alpha=0.1, gamma=0.99, epsilon=1.0):
        self.alpha = alpha # 학습률 (새로운 정보를 얼마나 받아들일까?)
        self.gamma = gamma # 할인율 (미래의 보상을 얼마나 중요시할까?)
        self.epsilon = epsilon # 탐험률 (새로운 길을 가볼까, 아는 길로 갈까?)
        self.epsilon_decay = 0.995
        self.min_epsilon = 0.01

        # Q-Table 초기화: 모든 칸에서 각 행동의 가치를 0으로 시작한다.
        self.q_table = {}
        for i in range(state_size):
            for j in range(state_size):
                self.q_table[(i, j)] = np.zeros(action_size)

    def get_action(self, obs):
        """Epsilon-Greedy 전략으로 다음 행동을 결정한다."""
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, 3) # 랜덤 탐험
        else:
            return np.argmax(self.q_table[obs]) # 아는 길 중 최선 선택

    def update(self, obs, action, reward, next_obs):
        """Q-Learning 공식을 사용하여 지식을 업데이트합니다."""
        old_value = self.q_table[obs][action]
        next_max = np.max(self.q_table[next_obs])

        # 공식: Q = Q + alpha * (보상 + 미래가치 - 현재가치)
        new_value = old_value + self.alpha * (reward + self.gamma * next_max - old_value)
        self.q_table[obs][action] = new_value

    def decay_epsilon(self):
        """학습이 진행될수록 탐험을 줄이고 아는 길로 가게 한다."""
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)

In [ ]:
# 1. 준비 단계
env = GridWorldEnv(size=5)
agent = QLearningAgent(state_size=5, action_size=4)
n_episodes = 300

# 2. 학습 루프 시작
for episode in range(n_episodes):
    # 환경 초기화
    obs, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        # [판단] 현재 상태에서 행동 선택
        action = agent.get_action(obs)

        # [실행] 행동을 취하고 피드백(보상 등) 받기
        next_obs, reward, terminated, truncated, info = env.step(action)

        # [학습] 에이전트의 지식 업데이트
        agent.update(obs, action, reward, next_obs)

        # 다음 단계 준비
        total_reward += reward
        obs = next_obs
        done = terminated or truncated

    # 한 게임이 끝날 때마다 탐험률 줄이기
    agent.decay_epsilon()

    if (episode + 1) % 50 == 0:
        print(f"에피소드: {episode+1}, 보상: {total_reward:.1f}, 탐험률: {agent.epsilon:.3f}")

print("\n--- 학습 완료! 이제 로봇이 길을 찾습니다. ---")

# 3. 테스트 (배운 실력 확인하기)
obs, info = env.reset()
env.render(agent.q_table) # 학습된 지능 시각화